In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
price_paid_df = pd.read_csv(r'C:\Users\Rekha\Desktop\Dissertation Datasets\PricePaid_Cleaned.csv')
epc_df = pd.read_csv(r'C:\Users\Rekha\Desktop\Dissertation Datasets\EPC_Cleaned.csv')

In [3]:
# Merge datasets based on common columns
hpp_df = price_paid_df.merge(epc_df, on=['Address', 'Postcode'], how='inner')

In [4]:
# Convert 'Transaction date' column to datetime
hpp_df['Transaction_date'] = pd.to_datetime(hpp_df['Transaction_date'])

# Convert 'Lodgement date' column to datetime
hpp_df['Lodgement_Date'] = pd.to_datetime(hpp_df['Lodgement_Date'])

# Calculate the difference between transaction date and lodgement date
hpp_df['Date_Difference'] = abs(hpp_df['Transaction_date'] - hpp_df['Lodgement_Date'])

# Find the nearest lodgement date for each transaction
nearest_lodgement = hpp_df.groupby(['Address', 'Postcode', 'Transaction_date'], as_index=False)['Date_Difference'].idxmin()

# Extract the index values from nearest_lodgement
index_values = nearest_lodgement['Date_Difference']

# Filter the merged DataFrame to include only the rows with the nearest lodgement date
hpp_df = hpp_df.loc[index_values]

# Drop the 'Date_Difference' column
hpp_df.drop(columns=['Date_Difference'], inplace=True)

In [5]:
# Replace empty strings with NaN
hpp_df.replace('', np.nan, inplace=True)

In [6]:
hpp_df.head()

,Transaction_date,Price_Paid,Address,Town/City_x,Postcode,Property_Type_x,Old/New,Freehold/Leasehold,District,County,Building_num,Lodgement_Date,Town/City_y,Property_Type_y,Floor_Area,Habitable_Rooms,Energy_Rating
590244,2020-08-25,780000,1 & 4 Haw Cote,Huddersfield,HD7 5XD,D,N,F,Kirklees,West Yorkshire,3367232578,2019-09-08,Huddersfield,House,196.0,7.0,D
714495,2022-10-27,125000,1 16Th Avenue,Hull,HU6 9JJ,S,N,F,City Of Kingston Upon Hull,City Of Kingston Upon Hull,8446903178,2013-09-05,Hull,House,73.0,5.0,D
736477,2022-12-06,115000,1 1St Avenue,Hull,HU6 9NE,S,N,F,City Of Kingston Upon Hull,City Of Kingston Upon Hull,10002965934,2022-09-19,Hull,House,75.0,5.0,D
570231,2020-12-18,70000,1 23Rd Avenue,Hull,HU6 8HP,S,N,F,City Of Kingston Upon Hull,City Of Kingston Upon Hull,9668157678,2019-11-02,Hull,House,57.0,4.0,D
726455,2022-04-19,100000,1 26Th Avenue,Hull,HU6 9SH,S,N,F,City Of Kingston Upon Hull,City Of Kingston Upon Hull,10001727195,2021-11-09,Hull,House,74.0,4.0,D


In [7]:
hpp_df.loc[hpp_df['Property_Type_x'] == 'O', 'Property_Type_x'] = hpp_df.loc[hpp_df['Property_Type_x'] == 'O', 'Property_Type_y']

In [8]:
hpp_df['Property_Type_x'].unique()

array(['D', 'S', 'T', 'House', 'Flat', 'F', 'Bungalow', 'Maisonette'],
      dtype=object)

In [9]:
hpp_df['Property_Type_y'].unique()

array(['House', 'Flat', 'Bungalow', 'Maisonette'], dtype=object)

In [10]:
hpp_df.shape

(615246, 17)

In [11]:
columns_drop = ['Town/City_y','Property_Type_y','Lodgement_Date']
hpp_df = hpp_df.drop(columns=columns_drop)

In [12]:
# Rename values in Property_Type_x
hpp_df['Property_Type_x'] = hpp_df['Property_Type_x'].replace({'Flat': 'F', 'Bungalow': 'B', 'Maisonette': 'M'})

# Drop rows with 'House' in Property_type_x since we can't conclude the type of the property with the value House
hpp_df = hpp_df[hpp_df['Property_Type_x'] != 'House']

In [13]:
hpp_df['Property_Type_x'].unique()

array(['D', 'S', 'T', 'F', 'B', 'M'], dtype=object)

In [14]:
hpp_df.shape

(609329, 14)

In [15]:
hpp_df = hpp_df.drop_duplicates()

In [16]:
# Rename columns in epc_df
hpp_df.rename(columns={'Town/City_x': 'Town/City','Property_Type_x':'Property_Type'}, inplace=True)

# Print the updated DataFrame
hpp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609329 entries, 590244 to 430553
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction_date    609329 non-null  datetime64[ns]
 1   Price_Paid          609329 non-null  int64         
 2   Address             609329 non-null  object        
 3   Town/City           609329 non-null  object        
 4   Postcode            609329 non-null  object        
 5   Property_Type       609329 non-null  object        
 6   Old/New             609329 non-null  object        
 7   Freehold/Leasehold  609329 non-null  object        
 8   District            609329 non-null  object        
 9   County              609329 non-null  object        
 10  Building_num        609329 non-null  int64         
 11  Floor_Area          609329 non-null  float64       
 12  Habitable_Rooms     527130 non-null  float64       
 13  Energy_Rating       6093

In [17]:
columns_to_strip = ['Address', 'Town/City', 'Postcode', 'Property_Type', 'Old/New', 'Freehold/Leasehold', 'District', 'County']

for column in columns_to_strip:
    hpp_df[column] = hpp_df[column].str.strip()

In [18]:
hpp_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 609329 entries, 590244 to 430553
Data columns (total 14 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Transaction_date    609329 non-null  datetime64[ns]
 1   Price_Paid          609329 non-null  int64         
 2   Address             609329 non-null  object        
 3   Town/City           609329 non-null  object        
 4   Postcode            609329 non-null  object        
 5   Property_Type       609329 non-null  object        
 6   Old/New             609329 non-null  object        
 7   Freehold/Leasehold  609329 non-null  object        
 8   District            609329 non-null  object        
 9   County              609329 non-null  object        
 10  Building_num        609329 non-null  int64         
 11  Floor_Area          609329 non-null  float64       
 12  Habitable_Rooms     527130 non-null  float64       
 13  Energy_Rating       6093

In [19]:
csv_file_path = r'C:\Users\Rekha\Desktop\Dissertation Datasets\House_Price.csv'

# Save the cleaned DataFrame to a CSV file
hpp_df.to_csv(csv_file_path, index=False)